In [1]:
import pandas as pd
import copy
from datetime import datetime as dt

### Списки выделяемых признаков

In [2]:
# кнопки мыши
mouse_buttons = [
    "Button.left",
#     "Button.right",
]

# специальные клавиши
special_keys = [
#     "Key.esc",
    "Key.tab",
#     "Key.caps_lock",
    "Key.shift",
    "Key.ctrl",
#     "Key.alt",
    "Key.cmd",
    "Key.space",
    "Key.enter",
    "Key.backspace",
]

# диграфы и триграфы - сочетания из 2-х и 3-х букв
eng_di = ["th", "he", "in", "er", "an", "re", "es", "on", "st", "nt", "en", "at", "ed", "nd", "to", "or", "ea"]
eng_tri = ["the", "and", "ing", "ent", "ion", "her", "for", "tha", "nth", "int", "ere", "tio", "ter", "est", "ers", "ati", "hat"]
ru_di = ["ст","ен","ов","но","ни","на","ра","ко","то","ро"] # ,"ан","ос","по","го","ер","од", "ре"]
ru_tri = ["ени","ост","ого","ств","ско","ста","ани","про","ест","тор"] # ,"льн","ова","ния","ние","при","енн","год"]

# признаки для специальных признаков и кнопок мыши
spec_features = {
    "dwell": [0, 0], # длительность нажатия
    "interval" : [0, 0], # промежуток между отпусканием текущей и нажатием следующей
    "flight": [0,0] # промежуток между нажатием текущей и нажатием следующей
}

# признаки для диграфов
di_features = {
    "dwell_first": [0,0], # длительность нажатия первой буквы
    "dwell_second": [0,0], # длительность нажатия второй буквы
    "interval": [0,0], # промежуток между отпусканием первой и нажатием второй буквы
    "flight": [0,0], # промежуток между нажатием первой и нажатием второй буквы
    "up_to_up": [0,0], # промежуток между отпусканием первой и отпусканием второй
    "latency": [0,0], # промежуток между нажатием первой и отпусканием второй
}

# признаки для триграфов
tri_features = {
    "dwell_first": [0,0], # длительность нажатия первой буквы
    "dwell_second": [0,0], # длительность нажатия второй буквы
    "dwell_third": [0,0], # длительность нажатия третьей буквы
    "interval_first": [0,0], # промежуток между отпусканием первой и нажатием второй буквы
    "interval_second": [0,0], # промежуток между отпусканием второй и нажатием третьей буквы
    "flight_first": [0,0], # промежуток между нажатием первой и нажатием второй буквы
    "flight_second": [0,0], # промежуток между нажатием второй и нажатием третьей буквы    
    "up_to_up_first": [0,0], # промежуток между отпусканием первой и отпусканием второй
    "up_to_up_second": [0,0], # промежуток между отпусканием второй и отпусканием третьей
    "latency": [0,0], # промежуток между нажатием первой и отпусканием третьей
}

### Выделение признаков

In [3]:
# здесь будут храниться все вышеприведённые признаки
features = {}

In [4]:
class FeatureProcessor:
    # вычисление признаков для кнопок мыши и униграфов
    def mouse_and_special_keys(self, df, events):
        for event in events:
            if event not in features: # в словаре признаков пока не выделена память под эту кнопку
                features[event] = copy.deepcopy(spec_features) # выделяем память
            for i, row in df.iterrows():
                if row[1] == "press" and row[2] == event:
                    press = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f') # фиксируем время нажатия кнопки
                    # время отпускания нажатой кнопки 
                    # по умолчанию задаётся значением press, 
                    # так как release для последнего события в датасете может отсутствовать
                    release = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                    # время нажатия кнопки после отпускания текущей
                    # по умолчанию задаётся значением press, 
                    # так как press следующей кнопки для последнего события в датасете может отсутствовать
                    press_next = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                    j = 1
                    while True: # ищем время отпускания кнопки
                        try: # пробуем обратиться по индексу i + j
                            next_row = df.iloc[i + j]
                            if next_row[1] == "release" and next_row[2] == event:
                                # фиксируем время отпускания текущей кнопки
                                release = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                try: # пробуем обратиться по индексу i +j + 1
                                    # фиксируем время нажатия следующей кнопки
                                    press_next = dt.strptime(df.iloc[i + j + 1][0], '%Y-%m-%d %H:%M:%S,%f')
                                except IndexError: # в случае отсутствия такого индекса, т.е. в случае выхода за границу датафрейма
                                    # фиксируем время нажатия следующей кнопки, как время отпускания текущей
                                    press_next = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                finally:
                                    # выходим из вспомогательного цикла в любом случае, так как release был найден
                                    break
                            else:
                                j+=1 # переход к следующей строке
                        except IndexError:
                            # выходим из вспомогательного цикла
                            break
                    # считаем признаки
                    # dwell
                    features[event]["dwell"][0] += (release - press).microseconds // 1000 # прибавляем длительность
                    features[event]["dwell"][1] += 1 # увеличиваем количество обработанных кнопок event
                    # interval
                    features[event]["interval"][0] += (press_next - release).microseconds // 1000 # прибавляем длительность
                    features[event]["interval"][1] += 1 # увеличиваем количество обработанных кнопок event
                    # flight
                    features[event]["flight"][0] += (press_next - press).microseconds // 1000 # прибавляем длительность
                    features[event]["flight"][1] += 1 # увеличиваем количество обработанных кнопок event
                
    # вычисление признаков для диграфов
    def digraph_features(self, df, events):
        for event in events:
            if event not in features: # в словаре признаков пока не выделена память под эту кнопку
                features[event] = copy.deepcopy(di_features) # выделяем память
            k = 0 # отвечает за индекс текущего символа в диграфе
            first_press = ""
            first_release = ""
            for i, row in df.iterrows():
                if row[1] == "press":
                    if row[2][1:-1].lower() == event[k]:
                        press = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f') # фиксируем время нажатия кнопки
                        # время отпускания нажатой кнопки 
                        # по умолчанию задаётся значением press, 
                        # так как release для последнего события в датасете может отсутствовать
                        release = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                        # время нажатия кнопки после отпускания текущей
                        # по умолчанию задаётся значением press, 
                        # так как press следующей кнопки для последнего события в датасете может отсутствовать
                        press_next = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                        j = 1
                        while True: # ищем время отпускания кнопки
                            try: # пробуем обратиться по индексу i + j
                                next_row = df.iloc[i + j]
                                if next_row[1] == "release" and next_row[2][1:-1] == event[k]:
                                    # фиксируем время отпускания текущей кнопки
                                    release = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                    try: # пробуем обратиться по индексу i +j + 1
                                        # фиксируем время нажатия следующей кнопки
                                        press_next = dt.strptime(df.iloc[i + j + 1][0], '%Y-%m-%d %H:%M:%S,%f')
                                    except IndexError: # в случае отсутствия такого индекса, т.е. в случае выхода за границу датафрейма
                                        # фиксируем время нажатия следующей кнопки, как время отпускания текущей
                                        press_next = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                    finally:
                                        # выходим из вспомогательного цикла в любом случае, так как release был найден
                                        break
                                else:
                                    j+=1 # переход к следующей строке
                            except IndexError:
                                # выходим из вспомогательного цикла
                                break
                        if k == 0:
                            # сохраняем момент нажатия и отпускания первой клавиши
                            first_press = press
                            first_release = release
                            k+=1
                        else:
                            # считаем признаки
                            # dwell_first
                            features[event]["dwell_first"][0] += (first_release - first_press).microseconds // 1000 # прибавляем длительность
                            features[event]["dwell_first"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # interval
                            features[event]["interval"][0] += (press - first_release).microseconds // 1000 # прибавляем длительность
                            features[event]["interval"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # flight
                            features[event]["flight"][0] += (press - first_press).microseconds // 1000 # прибавляем длительность
                            features[event]["flight"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # dwell_second
                            features[event]["dwell_second"][0] += (release - press).microseconds // 1000 # прибавляем длительность
                            features[event]["dwell_second"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # up_to_up
                            features[event]["up_to_up"][0] += (release - first_release).microseconds // 1000 # прибавляем длительность
                            features[event]["up_to_up"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # latency
                            features[event]["latency"][0] += (release - first_press).microseconds // 1000 # прибавляем длительность
                            features[event]["latency"][1] += 1 # увеличиваем количество обработанных кнопок event 
                            k = 0
                            first_press = ""
                            first_release = ""
                    else:
                        k = 0
                        first_press = ""
                        first_release = ""
    
    # вычисление признаков для триграфов
    def trigraph_features(self, df, events):
        for event in events:
            if event not in features: # в словаре признаков пока не выделена память под эту кнопку
                features[event] = copy.deepcopy(tri_features) # выделяем память
            k = 0 # отвечает за индекс текущего символа в диграфе
            first_press = ""
            first_release = ""
            second_press = ""
            seconds_release = ""
            for i, row in df.iterrows():
                if row[1] == "press":
                    if row[2][1:-1].lower() == event[k]:
                        press = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f') # фиксируем время нажатия кнопки
                        # время отпускания нажатой кнопки 
                        # по умолчанию задаётся значением press, 
                        # так как release для последнего события в датасете может отсутствовать
                        release = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                        # время нажатия кнопки после отпускания текущей
                        # по умолчанию задаётся значением press, 
                        # так как press следующей кнопки для последнего события в датасете может отсутствовать
                        press_next = dt.strptime(row[0], '%Y-%m-%d %H:%M:%S,%f')
                        j = 1
                        while True: # ищем время отпускания кнопки
                            try: # пробуем обратиться по индексу i + j
                                next_row = df.iloc[i + j]
                                if next_row[1] == "release" and next_row[2][1:-1] == event[k]:
                                    # фиксируем время отпускания текущей кнопки
                                    release = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                    try: # пробуем обратиться по индексу i +j + 1
                                        # фиксируем время нажатия следующей кнопки
                                        press_next = dt.strptime(df.iloc[i + j + 1][0], '%Y-%m-%d %H:%M:%S,%f')
                                    except IndexError: # в случае отсутствия такого индекса, т.е. в случае выхода за границу датафрейма
                                        # фиксируем время нажатия следующей кнопки, как время отпускания текущей
                                        press_next = dt.strptime(next_row[0], '%Y-%m-%d %H:%M:%S,%f')
                                    finally:
                                        # выходим из вспомогательного цикла в любом случае, так как release был найден
                                        break
                                else:
                                    j+=1 # переход к следующей строке
                            except IndexError:
                                # выходим из вспомогательного цикла
                                break
                        if k == 0:
                            # сохраняем момент нажатия и отпускания первой клавиши
                            first_press = press
                            first_release = release
                            k+=1
                        elif k == 1:
                            # сохраняем момент нажатия и отпускания первой клавиши
                            second_press = press
                            second_release = release
                            k+=1
                        else:
                            # считаем признаки
                            # dwell_first
                            features[event]["dwell_first"][0] += (first_release - first_press).microseconds // 1000 # прибавляем длительность
                            features[event]["dwell_first"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # interval_first
                            features[event]["interval_first"][0] += (second_press - first_release).microseconds // 1000 # прибавляем длительность
                            features[event]["interval_first"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # flight_first
                            features[event]["flight_first"][0] += (second_press - first_press).microseconds // 1000 # прибавляем длительность
                            features[event]["flight_first"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # up_to_up_first
                            features[event]["up_to_up_first"][0] += (second_release - first_release).microseconds // 1000 # прибавляем длительность
                            features[event]["up_to_up_first"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # dwell_second
                            features[event]["dwell_second"][0] += (second_release - second_press).microseconds // 1000 # прибавляем длительность
                            features[event]["dwell_second"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # interval_second
                            features[event]["interval_second"][0] += (press - second_release).microseconds // 1000 # прибавляем длительность
                            features[event]["interval_second"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # flight_second
                            features[event]["flight_second"][0] += (press - second_press).microseconds // 1000 # прибавляем длительность
                            features[event]["flight_second"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # up_to_up_second
                            features[event]["up_to_up_second"][0] += (release - second_release).microseconds // 1000 # прибавляем длительность
                            features[event]["up_to_up_second"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # dwell_third
                            features[event]["dwell_third"][0] += (release - press).microseconds // 1000 # прибавляем длительность
                            features[event]["dwell_third"][1] += 1 # увеличиваем количество обработанных кнопок event
                            # latency
                            features[event]["latency"][0] += (release - first_press).microseconds // 1000 # прибавляем длительность
                            features[event]["latency"][1] += 1 # увеличиваем количество обработанных кнопок event 
                            k = 0
                            first_press = ""
                            first_release = ""
                            second_press = ""
                            second_release = ""
                    else:
                        k = 0
                        first_press = ""
                        first_release = ""
                        second_press = ""
                        second_release = ""
    
    # расчёт средних показателей и запись в файл с признаками
    def calc_average(self, is_insider):
        averaged = [] # список для хранения усреднённых признаков
        # расчёт средних показателей
        for k, v in features.items():
            if k in special_keys: # для специальных признаков кроме временных показателей
                    averaged.append([k, value[1]]) # сохраняются ещё и частотные показатели
            for key, value in v.items():
                averaged.append([k + "_" + key, round(value[0] / value[1], 2) if value[1] != 0 else 0])
                
        # открываем файл с признаками для проверки существования заголовков
        with open("../data/features.csv", "r") as f:
            a = f.readlines()
            f.close()
            
        # открываем файл с признаками для записи заголовков и последующей записи признаков
        with open("../data/features.csv", "a") as f:
            if len(a) == 0: # в файле признаков нет заголовка
                headers = "" # зашоловки, объединённые в строку
                for i in range(len(averaged)):
                    if i != len(averaged) - 1:
                        headers += averaged[i][0] + "|"
                    else: # не добавляем "|" для последнего заголовка
                        headers += averaged[i][0] + "|is_insider\n"
                f.write(headers)
            
            averaged_features = "" # усреднённые признаки, объединённые в строку
            for i in range(len(averaged)):
                if i != len(averaged) - 1:
                    averaged_features += str(averaged[i][1]) + "|"
                else: # не добавляем "|" для последнего заголовка
                    averaged_features += str(averaged[i][1]) + "|" + str(1 if is_insider else 0) + "\n"
            f.write(averaged_features)
            

In [5]:
pt = pd.read_csv("../data/participants.csv", sep="|", header=None)
fp = FeatureProcessor()
# подсчёт признаков для сценариев без и с внутренними нарушителями
for i in ["non_insider","insider"]:
    # пробегаемся по списку участников
    for _, row in pt.iterrows():
        # для каждого из 3-х сценариев
        for j in range(1,4):
            # очистка словаря для хранения признаков
            features.clear() 
            # считывание данных
            kb = pd.read_csv('../data/keyboard_' + row[0] + '_' + i + '_' + str(j) + '.csv', sep='|', header=None)
            ms = pd.read_csv('../data/mouse_' + row[0] + '_' + i + '_' + str(j) + '.csv', sep='|', header=None)
            # подсчёт признаков
            fp.mouse_and_special_keys(ms, mouse_buttons)
            fp.mouse_and_special_keys(kb, special_keys)
            fp.digraph_features(kb, ru_di)
            # fp.digraph_features(kb, eng_di)
            fp.trigraph_features(kb, ru_tri)
            # fp.trigraph_features(kb, eng_tri)
            fp.calc_average(False if i == "non_insider" else True)